In [1]:
!nvidia-smi

Sat May  1 15:59:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 15.4MB/s 
     |████████████████████████████████| 901kB 68.8MB/s 
     |████████████████████████████████| 3.3MB 65.4MB/s 


In [3]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-01 15:59:19--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-01 15:59:20--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7090b42da1f410a5e35f92dea3.dl.dropboxusercontent.com/cd/0/inline/BNozSQ9C0S6QYQB_RLAX2w3qV6yYL3zMQSxAXfblo_gel7elXbElhMneUCeFTSBZXFarfinERShZlDqAeIZ6zX9U2YBl1tVcZgaMvpi61idYhfrGAGinp0uaqUGMswpHGGLfFp022m6W-eMehEvCIEY5/file# [following]
--2021-05-01 15:59:20--  https://uc7090b42da1f410a5e35f92dea3.dl.dropboxusercontent.com/cd/0/inline/BNozSQ9C0S6QYQB_RLAX2w3qV6yYL3zMQSxAXfblo_gel7elXbElhMneUCeFT

In [4]:
!wget https://www.dropbox.com/sh/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa?dl=0 -O model.zip
!unzip -o model.zip -d model

--2021-05-01 16:00:49--  https://www.dropbox.com/sh/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa [following]
--2021-05-01 16:00:49--  https://www.dropbox.com/sh/raw/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc6181e1e30584d761d0c910c2f.dl.dropboxusercontent.com/zip_download_get/AxHu9aPEcanSU1_XKXPoMGMIxd-F4vEDTLv1cDdzbXY9w1CdlpoeztCuGijImlhz7OtvhohqHFNY48JtwrvTin7D_m6nMTKgZ72eXO5XOYrRvQ [following]
--2021-05-01 16:00:50--  https://ucc6181e1e30584d761d0c910c2f.dl.dropboxusercontent.com/zip_download_get/AxHu9aPEcanSU1_XKXPoMGMIxd-F4vEDTLv1cDdzbXY9w1CdlpoeztCuGijImlhz7OtvhohqHFNY48

In [5]:
import pickle
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [6]:
data = pd.read_pickle(r'grouped_data.pickle')

In [7]:
texts = []
bos = "<BOS> "
eos = " <EOS> "

for key in list(data.keys()):
    post = data[key][0]
    comments = data[key][1]
    title = post['title']
    try:
        text = post['selftext']
    except:
        text = ""
    
    full_post = title + " " + text
    for comment in comments:
        if comment["score"] >= 1 and comment["body"] != "[deleted]":
            texts.append(full_post + bos + comment["body"]  + eos )

train_sample, test_sample = train_test_split(texts, train_size=0.1, test_size=0.025, shuffle=True, stratify=None)

print(len(train_sample), len(test_sample))

with open('train_text.txt', 'w', encoding='utf8') as f:
    for item in train_sample:
        f.write("%s\n" % item)

with open('test_text.txt', 'w', encoding='utf8') as f:
    for item in test_sample:
        f.write("%s\n" % item)

155494 38874


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_text.txt'
test_path = 'test_text.txt'

In [9]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (28286044 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
print('Size of training set:', len(train_dataset))
print('Size of test set:', len(test_dataset))

Size of training set: 220984
Size of test set: 55468


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [11]:
model = AutoModelWithLMHead.from_pretrained("model")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [12]:
training_args = TrainingArguments(
    output_dir="./gpt2-CC", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler,
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [13]:
trainer.train()

Step,Training Loss
500,2.888800
1000,2.914000
1500,2.905200
2000,2.897600
2500,2.885800
3000,2.893500
3500,2.884500
4000,2.878300
4500,2.871600
5000,2.878900


TrainOutput(global_step=27624, training_loss=2.78213651111911, metrics={'train_runtime': 9664.4378, 'train_samples_per_second': 2.858, 'total_flos': 4.223878123172659e+16, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': -300982272, 'init_mem_gpu_alloc_delta': 511148032, 'init_mem_cpu_peaked_delta': 422727680, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -10648330240, 'train_mem_gpu_alloc_delta': 1509547520, 'train_mem_cpu_peaked_delta': 10662502400, 'train_mem_gpu_peaked_delta': 4969395200})

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [14]:
trainer.save_model()

In [15]:
trainer.save_model(output_dir='model_trained')

In [16]:
!zip -r model_trained.zip model_trained/

  adding: model_trained/ (stored 0%)
  adding: model_trained/training_args.bin (deflated 46%)
  adding: model_trained/config.json (deflated 50%)
  adding: model_trained/pytorch_model.bin (deflated 9%)


In [17]:
!ls

cached_lm_GPT2TokenizerFast_128_test_text.txt	     model_trained
cached_lm_GPT2TokenizerFast_128_test_text.txt.lock   model_trained.zip
cached_lm_GPT2TokenizerFast_128_train_text.txt	     model.zip
cached_lm_GPT2TokenizerFast_128_train_text.txt.lock  runs
gpt2-CC						     sample_data
grouped_data.pickle				     test_text.txt
model						     train_text.txt
